# **Wine Cellars Web Scraping Program**

---

In [4]:
# Importing the necessary libraries:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException

In [ ]:
!pip install webdriver_manager

**Scraping the wine scoring scale data from the "https://www.wine-searcher.com/critics-8-cellartracker?page=1" website**

In [26]:
# Scraping the scoring scale data:
webpage_scoring_scale = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
webpage_scoring_scale.get("https://www.wine-searcher.com/critics-8-cellartracker?page=1")
content_page = webpage_scoring_scale.page_source
result_scoring_scale = BeautifulSoup(content_page, 'html.parser')
prd_page1 = result_scoring_scale.find_all('div', {'class':'heading-sm'})

scoring_scale = []
for score_scale_item in prd_page1:
    score_scale_item.ul
    for item in score_scale_item.ul:
        scoring_scale.append(item.string)

for i in scoring_scale:
    if i == '\n':
        scoring_scale.remove(i)

scoring_scale

['98–100 – Extraordinary',
 '94–97 – Outstanding',
 '90–93 – Very good',
 '86–89 – Good',
 '80–85 – Average',
 '70–79 – Below average',
 '50–69 – Avoid']

In [27]:
# Creating the original dataframe for the scaling scale:
df_scoring_scale = pd.DataFrame(scoring_scale, columns=['Scoring_Scale'])
df_scoring_scale

,Scoring_Scale
0,98–100 – Extraordinary
1,94–97 – Outstanding
2,90–93 – Very good
3,86–89 – Good
4,80–85 – Average
5,70–79 – Below average
6,50–69 – Avoid


In [28]:
# Splitting the df_scoring_scale dataframe into 2 columns, 'Scoring' and 'Scale':

scoring = []
scale = []

for i in list(df_scoring_scale['Scoring_Scale']):
    rep = i.replace(i[6:], '')
    stp = rep.rstrip()
    scoring.append(stp)

for j in list(df_scoring_scale['Scoring_Scale']):
    rep = j.replace(j[:8], '')
    stp = rep.lstrip()
    scale.append(stp)

In [29]:
scoring

['98–100', '94–97', '90–93', '86–89', '80–85', '70–79', '50–69']

In [30]:
scale

['Extraordinary',
 'Outstanding',
 'Very good',
 'Good',
 'Average',
 'Below average',
 'Avoid']

In [35]:
# Updating the scoring scale dataframe:
df_scoring_scale_updated = pd.DataFrame(zip(scoring, scale), columns=['Scoring', 'Scale'])
df_scoring_scale_updated

,Scoring,Scale
0,98–100,Extraordinary
1,94–97,Outstanding
2,90–93,Very good
3,86–89,Good
4,80–85,Average
5,70–79,Below average
6,50–69,Avoid


In [36]:
df_scoring_scale_updated.to_csv('C:\\Users\\cesar\\OneDrive\\Documents\\Cesar documents\\Data Science Projects\\Wine Ratings Analysis\\Dataset\\scoring-scale.csv', index=False)

**Scraping the data needed from the "https://www.winemag.com/ratings/" website**

In [ ]:
# Creating and installing the webdriver using Selenium:
webpage = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Loading the initial webpage:
webpage.get("https://www.winemag.com/ratings/?s=&drink_type=wine&sort_by=pub_date_web&sort_dir=desc")

# Creating a counter for each page:
counter = 1

# Creating the lists that will hold the data scraped:
product_name = []
appellation = []
rating = []
price = []

# Scraping the first page:
content_first_page = webpage.page_source
result = BeautifulSoup(content_first_page, 'html.parser')
product_content = result.find_all('li', {'class': 'review-item'})

for item in product_content:
    product_name.append(item.h3.string)
    appellation.append(item.find('span', {'class': 'appellation'}))
    rating.append(item.find('span', {'class': 'rating'}))
    price.append(item.find('span', {'class': 'price'}))
      
# Loading the second page and waiting for 30 seconds until the scraping of the second page is done as per below:
WebDriverWait(webpage, 30).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/section/section/div[1]/div[1]/div[7]/section/div[3]/div/div/span[2]"))).click()
counter += 1

# Looping each page and scraping it:
while webpage.find_element(By.XPATH, '/html/body/div[1]/div/section/section/div[1]/div[1]/div[7]/section/div[3]/div/div/span[2]') and counter <= 1850:
    try:
        content = webpage.page_source
        result = BeautifulSoup(content, 'html.parser')
        product_content = result.find_all('li', {'class': 'review-item'})
        
        for item in product_content:
            product_name.append(item.h3.string)
            appellation.append(item.find('span', {'class': 'appellation'}))
            rating.append(item.find('span', {'class': 'rating'}))
            price.append(item.find('span', {'class': 'price'}))
    
        # Since the page is dynamic and has notifications poping-up, it is imperative to find the button and execute it; in this case a click of this button:
        next_button = webpage.find_element(By.XPATH, "/html/body/div[1]/div/section/section/div[1]/div[1]/div[7]/section/div[3]/div/div/span[2]")
        webpage.execute_script("arguments[0].click();", next_button)
        counter += 1
        time.sleep(5)
    except:
        print('Unexpected error occured on this page! Loading the next page...')
        next_button = webpage.find_element(By.XPATH, "/html/body/div[1]/div/section/section/div[1]/div[1]/div[7]/section/div[3]/div/div/span[2]")
        webpage.execute_script("arguments[0].click();", next_button)
        counter += 1

In [ ]:
# Checking the data in the 'product_name':
product_name

In [ ]:
# Appending the 'appellation' and printing the results:
appellation_list = []
for i in appellation:
    appellation_list.append(i.string)
    print(i.string)

In [ ]:
# Appending the 'rating' and printing the results:
rating_list = []
for i in rating:
    rating_list.append(i.strong.string)
    print(i.strong.string)

In [ ]:
# Appending the 'price_list' and printing the results:
price_list = []
for i in price:
    price_list.append(i.string)
    print(i.string)

In [14]:
df = pd.DataFrame(zip(product_name, appellation_list, rating_list, price_list), columns=['Product_Name', 'Appellation', 'Rating', 'Price' ])
df

,Product_Name,Appellation,Rating,Price
0,Charles Heidsieck NV Champagne Charlie Brut (...,Champagne,100,$700
1,Stag's Leap Wine Cellars 2019 Fay Estate Grown...,Napa,100,$155
2,Marqués de Riscal 2016 150 Aniversario Gran Re...,Northern Spain,99,$60
3,Salon 2012 Le Mesnil Blanc de Blancs Brut Char...,Champagne,99,$992
4,Philipponnat 2013 Clos des Goisses Extra Brut ...,Champagne,99,$320
...,...,...,...,...
11099,Bodegas Franco-Españolas 2016 Royal Reserva Te...,Northern Spain,92,$45
11100,Cedarville 2019 Estate Grenache (El Dorado),Sierra Foothills,92,$36
11101,Doña Paula 2018 Alluvia Parcel Bush Vines Malb...,Mendoza Province,92,$100
11102,El Enemigo 2017 Gran Enemigo El Cepillo Single...,Mendoza Province,92,$100


In [17]:
df.to_csv("C:\\Users\\cesar\\OneDrive\\Documents\\Cesar documents\\Data Science Projects\\Wine Ratings Analysis\\Dataset\\wine-data.csv", index=False)

In [16]:
# Getting rid of the redundant records using 'groupby':

df2 = df.groupby(['Product_Name', 'Appellation', 'Rating', 'Price']).count().reset_index()
df2

,Product_Name,Appellation,Rating,Price
0,1+1=3 NV Cygnus Sador Brut Nature Reserva Spar...,Catalonia,89,$22
1,"10,000 Hours 2019 Red (Red Mountain)",Columbia Valley,88,$35
2,"10,000 Hours 2019 Syrah (Red Mountain)",Columbia Valley,90,$35
3,1000 Stories 2020 Bourbon Barrel Aged Batch No...,California Other,88,$19
4,12 Linajes 2017 Reserva Tempranillo (Ribera d...,Northern Spain,94,$65
...,...,...,...,...
10206,Zuccardi 2020 Poligonos Malbec (Paraje Altamira),Mendoza Province,90,$30
10207,Zuccardi 2020 Poligonos San Pablo Cabernet Fra...,Mendoza Province,91,$30
10208,Zuccardi 2020 Q Malbec (Uco Valley),Mendoza Province,90,$20
10209,Ökonomierat Rebholz 2015 Hansjörg Rebholz Sieb...,Germany,94,$155


In [48]:
# Getting rid of the currency in the 'Price' column and changing its name to 'Price_usd'

Price_usd = []
for i in df2['Price']:
    Price_usd.append(i.strip('$'))

df3 = pd.DataFrame(Price_usd, columns=['Price_usd'])

df4 = pd.DataFrame(zip(product_name, appellation_list, rating_list, Price_usd), columns=['Product_Name', 'Appellation', 'Rating', 'Price_usd'])
df4

,Product_Name,Appellation,Rating,Price_usd
0,Charles Heidsieck NV Champagne Charlie Brut (...,Champagne,100,20
1,Stag's Leap Wine Cellars 2019 Fay Estate Grown...,Napa,100,22
2,Salon 2012 Le Mesnil Blanc de Blancs Brut Char...,Champagne,99,35
3,Philipponnat 2013 Clos des Goisses Extra Brut ...,Champagne,99,35
4,Stag's Leap Wine Cellars 2019 S.L.V. Estate Gr...,Napa,99,19
...,...,...,...,...
8522,Bruliam 2019 Sangacimo Roberts Road Pinot Noir...,Sonoma,90,130
8523,Brutocao 2015 Bliss Vineyard Zinfandel (Mendoc...,Mendocino County,90,30
8524,Brutocao 2018 Hopland Ranches Estate Cabernet ...,Mendocino County,90,30
8525,Kaiken 2018 Obertura Cabernet Franc (Mendoza),Mendoza Province,90,30


**Uploading the csv file into Azure Blob Storage**

In [1]:
!pip install azure-storage-blob

In [2]:
from azure.storage.blob import BlobServiceClient

In [5]:
# Reading the csv file into a pandas dataframe:
csv_df = pd.read_csv('C:\\Users\\cesar\\OneDrive\\Documents\\Cesar documents\\Data Science Projects\\Wine Ratings Analysis\\Dataset\\wine-data.csv')

# Creating a string object that holds the converted pandas dataframe into a csv:
csv_file = csv_df.to_csv(index=False, encoding='utf-8')

In [6]:
# Creating the blob service object in order to connect to Azure Blob Storage:
blob_service = BlobServiceClient.from_connection_string(
   "DefaultEndpointsProtocol=https;AccountName=**********;AccountKey=***********;EndpointSuffix=core.windows.net")

# Defining the container and the blob that we want to create inside the container: 
container_client = blob_service.get_container_client('datacontainer')
blob_client = blob_service.get_blob_client(container='datacontainer', blob='wine-data.csv') 

# Uploading the csv_file object as csv into Azure Blob Storage:
blob_client.upload_blob(csv_file, blob_type="BlockBlob")